# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model

opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [7]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [8]:
import cv2
inputVideo = ''
cap=cv2.VideoCapture(0) # 0 stands for very first webcam attach
#filename="output.avi"
#codec=cv2.VideoWriter_fourcc('m','p','4','v')#fourcc stands for four character code
#framerate=6
#resolution=(640,480)
    
#VideoFileOutput=cv2.VideoWriter(filename,codec,framerate, resolution)

with detection_graph.as_default():
  with tf.compat.v1.Session(graph=detection_graph) as sess:
    ret=True
    
    while (ret):
        ret, image_np=cap.read() 
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
          # Actual detection.
        (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
        
        
        for index, value in enumerate(classes[0]):
            ymin = boxes[0][index][0]
            xmin = boxes[0][index][1]
            ymax = boxes[0][index][2]
            xmax = boxes[0][index][3]
            class_name = (category_index.get(value)).get('name')
            widthvalue = int((xmax - xmin) / 2)  # width 길이
            heightvalue = int((ymax - ymin) / 2)  # height 길이
            if (class_name == "person" and scores[0, index] > 0.80) :
                print((xmin + xmax) / 2, (ymin + ymax) / 2)
                print("person"  + str(index) + "\n")
        #VideoFileOutput.write(image_np)
        cv2.imshow('live_detection', image_np)
        if cv2.waitKey(25) & 0xFF==ord('q'):
            break
            cv2.destroyAllWindows()
            cap.release()

0.5113248229026794 0.6618707776069641
person0

0.5351572632789612 0.6571267247200012
person0

0.5378181338310242 0.6587468981742859
person0

0.5374730825424194 0.6566405296325684
person0

0.5373334288597107 0.6531069278717041
person0

0.5329177975654602 0.6595147848129272
person0

0.5372880101203918 0.6540993452072144
person0

0.5340335369110107 0.6584168076515198
person0

0.5346253514289856 0.6560018658638
person0

0.5378702878952026 0.657172441482544
person0

0.5309567451477051 0.659488320350647
person0

0.5345711708068848 0.6595959663391113
person0

0.5289404988288879 0.6586418151855469
person0

0.5319594144821167 0.6598129272460938
person0

0.5269516110420227 0.6591804623603821
person0

0.5376387238502502 0.6554081439971924
person0

0.5366630554199219 0.6598170399665833
person0

0.5310395956039429 0.6589307188987732
person0

0.5304215550422668 0.662114679813385
person0

0.5360091924667358 0.6593400835990906
person0

0.5387712717056274 0.6599867939949036
person0

0.5269999504089355 

In [6]:
def set_baseline(x1, y1, x2, y2):
    return y2 - y1, x1 - x2, x2 * y1 - x1 * y2

In [ ]:
import cv2
car_index = [[[], []] for i in range(10)]

inputVideo = 'C:\\Users\\corallines\\Desktop\\VIDEO\\a3.mp4'
cap=cv2.VideoCapture(inputVideo) # 0 stands for very first webcam attach
#filename="output.avi"
#codec=cv2.VideoWriter_fourcc('m','p','4','v')#fourcc stands for four character code
#framerate=6
#resolution=(640,480)
    
#VideoFileOutput=cv2.VideoWriter(filename,codec,framerate, resolution)

with detection_graph.as_default():
  with tf.compat.v1.Session(graph=detection_graph) as sess:
    ret=True
    
    while (ret):
        ret, image_np=cap.read() 
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
          # Actual detection.
        (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
        
        for index, value in enumerate(classes[0]):
            ymin = boxes[0][index][0]
            xmin = boxes[0][index][1]
            ymax = boxes[0][index][2]
            xmax = boxes[0][index][3]
            class_name = (category_index.get(value)).get('name')
            widthvalue = int((xmax - xmin) / 2)  # width 길이
            heightvalue = int((ymax - ymin) / 2)  # height 길이
            if (class_name == "car" and scores[0, index] > 0.80) :
                x_center = (xmin + xmax) / 2
                y_center = (ymin + ymax) / 2
                car_index[index][0].append(x_center)
                car_index[index][1].append(y_center)
                            
        #VideoFileOutput.write(image_np)
        cv2.imshow('Sample Video Object Detection', image_np)
        if cv2.waitKey(25) & 0xFF==ord('q'):
            break
        cv2.destroyAllWindows()
        cap.release()

In [1]:
def get_distance_from_line(x1, y1, a, b, c):
    return abs(a*x1 + b*y1 + c) / (((a ** 2) + (b ** 2)) ** 0.5)